<a href="https://colab.research.google.com/github/Anjoom765/deeeplearnigcancerdete/blob/main/cancerproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -q tensorflow tensorflow-datasets

In [16]:
import tensorflow as tf
import tensorflow_datasets as tfds

import numpy as np
import matplotlib.pyplot as plt

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dense

In [ ]:
tfds.list_builders()


In [6]:
data=tfds.builder('colorectal_histology')
info=data.info
info

tfds.core.DatasetInfo(
    name='colorectal_histology',
    version=2.0.0,
    description='Classification of textures in colorectal cancer histology. Each example is a 150 x 150 x 3 RGB image of one of 8 classes.',
    homepage='https://zenodo.org/record/53169#.XGZemKwzbmG',
    features=FeaturesDict({
        'filename': Text(shape=(), dtype=tf.string),
        'image': Image(shape=(150, 150, 3), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=8),
    }),
    total_num_examples=5000,
    splits={
        'train': 5000,
    },
    supervised_keys=('image', 'label'),
    citation="""@article{kather2016multi,
      title={Multi-class texture analysis in colorectal cancer histology},
      author={Kather, Jakob Nikolas and Weis, Cleo-Aron and Bianconi, Francesco and Melchers, Susanne M and Schad, Lothar R and Gaiser, Timo and Marx, Alexander and Z{"o}llner, Frank Gerrit},
      journal={Scientific reports},
      volume={6},
      pages={27988},
      y

In [7]:
ds_train=tfds.load(name="colorectal_histology",split='train[:75%]')


ds_test=tfds.load(name="colorectal_histology",split='train[75%:100%]')




Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/colorectal_histology/2.0.0.incompleteW4UXS6/colorectal_histology-train.tfrecord


  0%|          | 0/5000 [00:00<?, ? examples/s]

Dataset colorectal_histology downloaded and prepared to /root/tensorflow_datasets/colorectal_histology/2.0.0. Subsequent calls will reuse this data.


In [8]:
train_images=np.array([example['image'].numpy()[:,:,0] for example in ds_train])
train_labels=np.array([example['label'].numpy() for example in ds_train])


test_images=np.array([example['image'].numpy()[:,:,0] for example in ds_test])

test_labels=np.array([example['label'].numpy() for example in ds_test])


In [14]:
type(train_images[0])
train_images.shape


(3750, 150, 150)

In [15]:
type(test_labels[0])
test_labels.shape


(1250,)

In [9]:
train_images=train_images.reshape(3750,150,150,1)
test_images=test_images.reshape(1250,150,150,1)

In [10]:
train_images=train_images.astype('float32')
test_images=test_images.astype('float32')


In [11]:
train_images/=255
test_images/=225

In [19]:
train_images[0].shape

(150, 150, 1)

MODELING A NN

In [ ]:
model=keras.Sequential([
                        keras.layers.Conv2D(128,3,activation='relu',input_shape=(150,150,1)),
                        keras.layers.MaxPooling2D((2,2)),
                        keras.layers.Conv2D(64,3,activation='relu'),
                        keras.layers.MaxPooling2D((2,2)),
                        keras.layers.Conv2D(32,3,activation='relu'),
                        keras.layers.MaxPooling2D((2,2)),
                        keras.layers.Flatten(),
                        keras.layers.Dense(128,activation='relu'),
                        keras.layers.Dense(64,activation='relu'),
                        keras.layers.Dense(8,activation='softmax')
]
)
model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=20, batch_size=32)


In [21]:
model.evaluate(test_images, test_labels)


40/40 [==============================] - 2s 49ms/step - loss: 1.5996 - accuracy: 0.6544


[1.5996103286743164, 0.6543999910354614]

##hyperparameter tuning


In [ ]:
pip install keras-tuner --upgrade


In [ ]:
import keras
import keras_tuner as kt
from kerastuner.tuners import RandomSearch

def build_model(hp):
   model=keras.Sequential()

   model.add( keras.layers.AveragePooling2D(6,3,input_shape=(150,150,1)))
   
   for i in range(hp.Int("conv_layers", min_value=0, max_value=3)): 
        model.add(keras.layers.Conv2D(hp.Choice(f"layer_{i}_filters",[16,32,64]),3,activation='relu',input_shape=(150,150,1)))
   model.add( keras.layers.MaxPooling2D(2,2))
   
   model.add(keras.layers.Flatten())
   
   model.add(keras.layers.Dense(hp.Choice('dense_layer', [64, 128, 256,512,1024]),activation='relu'))
   model.add(keras.layers.Dense(64,activation='relu'))
   model.add(keras.layers.Dense(8,activation='softmax'))

   model.compile(optimizer='adam',
               loss=keras.losses.SparseCategoricalCrossentropy(),
               metrics=['accuracy'])
   return model 


tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=32,
    overwrite=True,
    )

tuner.search(train_images,train_labels,validation_data=(test_images,test_labels),epochs=10,batch_size=32)


In [ ]:
best_model=tuner.get_best_models()[0]


In [25]:
best_model.evaluate(test_images,test_labels)

40/40 [==============================] - 2s 45ms/step - loss: 0.6770 - accuracy: 0.7336


[0.6770182251930237, 0.7336000204086304]

In [26]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 128)     1280      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        73792     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 32)        18464     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9248)              0

In [27]:
tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
dense_layer: 64
Score: 0.7336000204086304
Trial summary
Hyperparameters:
dense_layer: 1024
Score: 0.699999988079071
Trial summary
Hyperparameters:
dense_layer: 512
Score: 0.6696000099182129
Trial summary
Hyperparameters:
dense_layer: 128
Score: 0.6304000020027161
